<a href="https://colab.research.google.com/github/shinjangwoon/TIL/blob/master/scikit-learn/The_Predictive_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01_tabular_data_exploration

In [ ]:
import pandas as pd

php = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phpMawTba.csv')

- 이 데이터의 목표는 연령, 고용, 교육, 가족 정보 등과 같은 데이터에서 한 사람이 연간 5만 달러 이상을 벌고 있는지 예측하는 것입니다.

In [ ]:
php.info()

In [ ]:
php.head()

- class라는 열은 대상 변수(예측하려는 변수)입니다. 현재 가능한 클래스는 <=50k(저수익), >50k(고수익)입니다. 
- 따라서 결과 예측 문제는 이진 분류 문제이며, 다른 열을 모델의 입력 변수로 사용합니다.

In [ ]:
target_column = "class"
php[target_column].value_counts()

- 클래스는 약간 불균형합니다. 즉, 다른 클래스에 비해 하나 이상의 클래스 샘플이 더 많습니다. 
 
- 클래스 불균형은 실제로 자주 발생하며, 예측 모델을 구축할 때 특별한 기술이 필요할 수 있습니다.


In [ ]:
numerical_columns = [
    "age", "education-num", "capital-gain", "capital-loss",
    "hours-per-week"]
categorical_columns = [
    "workclass", "education", "marital-status", "occupation",
    "relationship", "race", "sex", "native-country"]
all_columns = numerical_columns + categorical_columns + [target_column]

php = php[all_columns]

In [ ]:
print(f"The dataset contains {php.shape[0]} samples and "
      f"{php.shape[1]} columns")

In [ ]:
print(f"The dataset contains {php.shape[1] - 1} features")

## 데이터 시각화

In [ ]:
_ = php.hist(figsize=(20, 14))

- 이전 셀에서는 _ = func() 패턴을 사용했습니다. 
- 이 경우에는 그다지 유용하지 않은 func() 의 출력을 표시하지 않기 위해 이 작업을 수행하였습니다. 
- 관례상, python에서 밑줄 변수는 우리가 관심이 없는 결과를 저장하기 위한 "쓰레기" 변수로 사용됩니다.

- "age" : 'age > 70' 이상이 이 데이터셋에 많지 않으므로 은퇴한 사람은 필터링 된 것을 알 수 있습니다.
- "hours-per-week" : 주당 40시간정도가 제일 높은 것을 알 수 있습니다.
- "capital-gain" and "capital-loss"는 0에 가깝습니다.

In [ ]:
php["sex"].value_counts()

In [ ]:
php['education'].value_counts()

In [ ]:
## education과 education-num의 관계
pd.crosstab(index=php['education'], columns=php['education-num']) 

- "education"과 "education-num"이 동일한 정보를 제공한다는 것을 보여줍니다.
ex) "education-num=2"는 "education=1st-4th"와 같습니다.
실제로 이는 정보 손실 없이 "education-num"을 제거할 수 있음을 의미합니다

* 중복(또는 높은 상관 관계)열이 있는 것은 기계 학습 알고리즘에 문제가 될 수 있습니다.

In [ ]:
'''
pairplot을 수행하고, class에 따라 각 변수가 어떻게 다른지 보여주는 것입니다.
대각선을 따라 플롯은 각 class에 대한 개별 변수의 분포를 보여줍니다.
비대각선의 플롯은 변수간의 흥미로운 상호 작용을 나타낼 수 있습니다.
'''
import seaborn as sns

n_sample_to_plot = 5000
columns = ["age", "education-num", "hours-per-week"]
_ = sns.pairplot(
    data=php[:n_sample_to_plot],
    vars=columns,
    hue=target_column,
    plot_kws={"alpha":0.2},
    height=3,
    diag_kind='hist',
    diag_kws={"bins":30},
)

## 결정 규칙 만들기
- 누가 고소득이고 누가 저소득인지 예측해보는 규칙을 만들어봅니다. 
- 예를 들어, 'hours-per-week'와 'age' 조합에 집중해봅니다.

In [ ]:
_ = sns.scatterplot(
    x="age",
    y="hours-per-week",
    data=php[:n_sample_to_plot],
    hue='class',
    alpha=0.5,

)

- 점은 dataset에서 'hours-per-week'와 'age'의 분포를 보여줍니다.
- 파란색 점은 저소득을 의미하며, 주황색 점은 고소득을 의미합니다.
- 이 plot은 pairplot에서 왼쪽 하단 플롯과 동일합니다.

- 이 plot에서 어떤 클래스를 예측해야 하는지 쉽게 결정할 수 있도록 
주로 단일 클래스를 포함하는 영역을 찾으려고 노력해야 합니다.


In [ ]:
import matplotlib.pyplot as plt

ax = sns.scatterplot(
    x="age",
    y='hours-per-week',
    data=php[:n_sample_to_plot],
    hue='class',
    alpha=0.5,
)
age_limit = 27
plt.axvline(x=age_limit, ymin=0, ymax=1, color="black", linestyle="--")

hours_per_week_limit = 40
plt.axhline(y=hours_per_week_limit, xmin=0.18, xmax=1, color="black", linestyle="--")

plt.annotate("<=50K", (17, 25), rotation=90, fontsize=35)
plt.annotate("<=50K", (35, 20), fontsize=35)
_ = plt.annotate("???", (45, 60), fontsize=35)

- 'age < 27' 인 왼쪽 영역은 예측이 저소득입니다.
 물론 많은 파란색 점이 있지만 주황색 점은 볼 수 없습니다.
- 'age > 27 and hours-per-week < 40'인 오른쪽 하단 영역은 저소득입니다.
많은 파란색 점이 있지만 조금씩 주황색 점도 보입니다. 
- 'age > 27 and hours-per-week > 40'인 오른쪽 상단 영역은 파란색 점과 주확색 점이 많습니다.
이 지역을 예측해야합니다. 

# 연습
- culmen 길이와 culmen 깊이의 두 가지 신체 측정을 ​​기반으로 펭귄 종을 예측해보기

*** sciki-learn 공부 중 제공 jupyter에서 해보았으나 csv가 가져오기 애매하여 코드만 적어둠


In [ ]:
# 데이터 불러오기
# import pandas as pd
# php = pd.read_csv('../datasets/penguins_classification.csv')
# php.head()

In [ ]:
# 수치형 컬럼과 자료형 컬럼 나누기
# numerical_columns = [
#     "Culmen Length(mm)", "Culmen Depth(mm)"
# ]
# catrgorical_columns = [
#     "Species"
# ]

In [ ]:
# 데이터 세트에서 펭귄 종류와 수 찾기
# php["Species"].value_counts()

In [ ]:
# 수치형 컬럼 시각화
# _ = php.hist(figsize=(10, 4))

In [ ]:
# class별 분포 시각화
# import seaborn as sns
# sns.pairplot(php, hue='Species', height=3)